In [3]:
import openai

# openai.base_url = 'http://222.29.156.145:8000/v1/'
# openai.api_key = 'key'
# model_name = "Qwen/Qwen2.5-32B-Instruct"
openai.base_url = 'https://api.deepseek.com'
openai.api_key = 'sk-54964e5c3b8c4998a74f7d3e35b618ac'
model_name = 'deepseek-chat'

resp = openai.chat.completions.create(
    model=model_name,
    messages=[
        {
            'role': 'user',
            'content': '知识图谱中的SPO三元组，SPO分别对应哪三者'
        },
    ]
)
print(resp.choices[0].message.content)

在知识图谱中，SPO三元组是表示知识的基本结构，其中：

- **S** 代表 **Subject（主语）**：表示知识中的主体或实体，通常是知识图谱中的一个节点。例如，“北京”可以是一个主语。
  
- **P** 代表 **Predicate（谓语）**：表示主语和宾语之间的关系或属性，通常是知识图谱中的一条边。例如，“是首都”可以是一个谓语。
  
- **O** 代表 **Object（宾语）**：表示知识中的客体或属性值，通常是知识图谱中的另一个节点。例如，“中国”可以是一个宾语。

因此，一个完整的SPO三元组可以表示为：**主语（S） - 谓语（P） - 宾语（O）**。例如：

- **北京（S） - 是首都（P） - 中国（O）**

这个三元组表示“北京是中国的首都”这一知识。


In [2]:
import json
from utils.parse_expr import *
data = json.load(open('data/processed_spice_data/dev_full.json', 'r'))
len(data)
# parser = ExprParser()


# for example in data:
#     example['new_sparql'] = parser.lisp_to_sparql(example['s_expression'])
# json.dump(data, open('data/processed_spice_data/new_train_full.json', 'w'), indent=4)

155734

In [ ]:
import json
from utils.parse_expr import lisp_to_sparql
data = json.load(open('data/processed_spice_data/train_full.json', 'r'))

for example in data:
    example['new_sparql'] = lisp_to_sparql(example['s_expression'])
json.dump(data, open('data/processed_spice_data/new_train_full.json', 'w'), indent=4)

In [ ]:
data = json.load(open('data/processed_spice_data/new_train_full.json', 'r'))

In [25]:
query = 'ASK { wd:Q597426 wdt:P106 wd:Q628099 .  wd:Q597426 wdt:P106 wd:Q937857 .  }'
query = 'prefix wd: <http://www.wikidata.org/entity/> prefix wdt: <http://www.wikidata.org/prop/direct/> ' + query
sparql.setQuery(query)
sparql.query().convert()

{'head': {'link': []}, 'boolean': True}

In [ ]:
from pymantic import sparql

server = sparql.SPARQLServer('http://localhost:9999/blazegraph/sparql')

query = 'SELECT ?x WHERE { wd:Q35 wdt:P361 ?x . ?x wdt:P31 wd:Q2221906 .  }'
query = 'prefix wd: <http://www.wikidata.org/entity/> prefix wdt: <http://www.wikidata.org/prop/direct/> ' + query
result = server.query(query)
print([binding[var]['value'].split('/')[-1] for binding in result['results']['bindings'] for var in binding])

In [1]:
prompt = r"""
        接下来请将问题转化为逻辑形式。
        这里的逻辑形式由以下的函数构成，其中大部分函数返回符合其条件的
        (JOIN predicate object), 返回集合，集合由所有使subject, predicate, object三元组为真的subject构成。这里和以下的集合可能含有重复元素。
        (JOIN (R predicate) subject), 返回集合，集合由所有使subject, predicate, object三元组为真的object构成, (R predicate)在这里表示关系的逆转。
        (AND set1 set2 ...), 返回若干集合的交集。
        (OR set1 set2 ...), 返回若干集合的并集。
        (DIFF set1 set2), 返回两个集合的差集set1-set2。
        (VALUES value1 value2 ...), 返回由v1,v2等值组成的集合。
        (DISTINCT set), 返回去重后的集合。
        (COUNT set), 返回集合的元素数目。
        (LT set value), 从set筛选出小于value的元素，作为新的集合返回. 类似地，LE, EQ, GE, GT分别表示小于等于，等于，大于等于，大于。
        (GROUP_COUNT set), 分组计数，返回python中的字典，键值分别为集合中的元素与其在集合中的数目。
        (GROUP_SUM group_count1 group_count2), 合并两个集合分组计数的结果，返回字典，键值分别为元素与其在两个集合中的总数目。
        (ARGMIN dictionary), 返回字典中对应值最小的键. 类似地，ARGMAX对应值最大的键。
        (ASK (s1 p1 o1) (s2 p2 o2) ...), 对若干三元组，判断其是否同时为真。
        """
print(prompt)


        接下来请将问题转化为逻辑形式。
        这里的逻辑形式由以下的函数构成，其中大部分函数返回符合其条件的
        (JOIN predicate object), 返回集合，集合由所有使subject, predicate, object三元组为真的subject构成。这里和以下的集合可能含有重复元素。
        (JOIN (R predicate) subject), 返回集合，集合由所有使subject, predicate, object三元组为真的object构成, (R predicate)在这里表示关系的逆转。
        (AND set1 set2 ...), 返回若干集合的交集。
        (OR set1 set2 ...), 返回若干集合的并集。
        (DIFF set1 set2), 返回两个集合的差集set1-set2。
        (VALUES value1 value2 ...), 返回由v1,v2等值组成的集合。
        (DISTINCT set), 返回去重后的集合。
        (COUNT set), 返回集合的元素数目。
        (LT set value), 从set筛选出小于value的元素，作为新的集合返回. 类似地，LE, EQ, GE, GT分别表示小于等于，等于，大于等于，大于。
        (GROUP_COUNT set), 分组计数，返回python中的字典，键值分别为集合中的元素与其在集合中的数目。
        (GROUP_SUM group_count1 group_count2), 合并两个集合分组计数的结果，返回字典，键值分别为元素与其在两个集合中的总数目。
        (ARGMIN dictionary), 返回字典中对应值最小的键. 类似地，ARGMAX对应值最大的键。
        (ASK (s1 p1 o1) (s2 p2 o2) ...), 对若干三元组，判断其是否同时为真。
        


In [1]:
import pickle
def sub_mid_to_fn(string, wikidata_mid_to_fn_dict):
    seg_list = string.split()
    for i in range(len(seg_list)):
        token = seg_list[i].strip(')(')
        if token.startswith('P') or token.startswith('Q'):
            fn = wikidata_mid_to_fn_dict.get(token, "unknown_entity")
            seg_list[i] = seg_list[i].replace(token, fn)
    new_string = ' '.join(seg_list)
    return new_string

with open('data/wikidata_mid_to_fn_dict.pickle', 'rb') as f:
    wikidata_mid_to_fn_dict = pickle.load(f)
len(wikidata_mid_to_fn_dict)

# sub_mid_to_fn('(ASK (Q1776510 child Francesco_of_Saluzzo) (Q1776510 P40 Gian_Gabriele_I_of_Saluzzo) (Q1776510 P40 Michele_Antonio,_Marquess_of_Saluzzo))', wikidata_mid_to_fn_dict)

610698

In [8]:
import json
data = json.load(open('data/processed_spice_data/dev_1000.json', 'r'))
for example in data:
    if example['s_expression'].startswith('(GROUP_'):
        print(example['question'])
        print(sub_mid_to_fn(example['s_expression'], wikidata_mid_to_fn_dict))
        print(example['sparql_query'])
        print()

Which administrative territories have a diplomatic relationship with around the same number of political territories as Mexico ? [CTX]
(GROUP_COUNT (AND (JOIN diplomatic_relation (JOIN instance_of political_territorial_entity)) (JOIN P31 administrative_territorial_entity_type)))
SELECT ?x  WITH { SELECT ?x (COUNT(*) AS ?tupcount) WHERE { ?x wdt:P530 ?y . ?x wdt:P31 wd:Q15617994 . ?y wdt:P31 wd:Q1048835 .  } GROUP BY ?x } AS  %tupcounts WITH {  SELECT DISTINCT ?x (0 AS ?tupcount) WHERE {  { { ?x wdt:P530 ?b . ?x wdt:P31 wd:Q15617994 .  } }  FILTER NOT EXISTS { ?x wdt:P530 ?y . ?x wdt:P31 wd:Q15617994 . ?y wdt:P31 wd:Q1048835 .  } }  } AS %zerotupcounts  WHERE { { SELECT ?x ?tupcount WHERE { INCLUDE %tupcounts } }  UNION { SELECT ?x ?tupcount WHERE { INCLUDE %zerotupcounts } }  }

Which administrative territories are the countries of origin of around the same number of literary works or television programs as Bulgaria ? [CTX]
(GROUP_SUM (GROUP_COUNT (AND (JOIN (R country_of_origin) (JOIN

In [ ]:
with open('../wikidata.ttl') as f:
    with open('data/wikidata_mid_to_fn.txt', 'w') as out_f:
        # for i in range(1000):
        while True:
            line = f.readline()
            if line.startswith('PREFIX'):
                continue
            seg_list = line.split(' ', 2)
            if seg_list[1] != 'rdfs:label':
                break
            mid = seg_list[0][3:]
            fn = seg_list[2].strip('" .\n')
            print(mid, fn, file=out_f)

#             query = 'SELECT ?fn WHERE { wd:' + mid + ' rdfs:label ?fn }'
#             query = 'prefix wd: <http://www.wikidata.org/entity/> prefix wdt: <http://www.wikidata.org/prop/direct/> ' + query
#             result = server.query(query)
#             fn = [binding[var]['value'].split('/')[-1] for binding in result['results']['bindings'] for var in binding]
#             assert(len(fn) == 1)
#             wikidata_mid_to_fn_dict[mid] = fn[0]
# pickle.dump(wikidata_mid_to_fn_dict, open('data/wikidata_mid_to_fn_dict.pickle', 'wb'))


In [4]:
def struct_of_exp(exp):
    ans = []
    for token in exp.split():
        if token[0] == '(':
            ans.append(token)
        elif token[-1] == ')':
            n = token.count(')')
            ans.append('x')
            ans.append(token[-n:])
        else:    
            ans.append('x')
    return ans

exp = '(GT (GROUP_SUM (GROUP_COUNT (AND (JOIN (R extends_upon) (JOIN instance_of literary_work)) (JOIN instance_of work_of_art))) (GROUP_COUNT (AND (JOIN (R extends_upon) (JOIN instance_of collectable)) (JOIN instance_of work_of_art)))) (OR (AND (JOIN extends_upon The_World_of_Henry_Orient) (JOIN instance_of literary_work)) (AND (JOIN extends_upon The_World_of_Henry_Orient) (JOIN instance_of collectable))))'

struct_of_exp(exp)

['(GT',
 '(GROUP_SUM',
 '(GROUP_COUNT',
 '(AND',
 '(JOIN',
 '(R',
 'x',
 ')',
 '(JOIN',
 'x',
 'x',
 '))',
 '(JOIN',
 'x',
 'x',
 ')))',
 '(GROUP_COUNT',
 '(AND',
 '(JOIN',
 '(R',
 'x',
 ')',
 '(JOIN',
 'x',
 'x',
 '))',
 '(JOIN',
 'x',
 'x',
 '))))',
 '(OR',
 '(AND',
 '(JOIN',
 'x',
 'x',
 ')',
 '(JOIN',
 'x',
 'x',
 '))',
 '(AND',
 '(JOIN',
 'x',
 'x',
 ')',
 '(JOIN',
 'x',
 'x',
 '))))']